# 1. Problem Statement

#### Business Problem Overview
In the telecom industry, customers are able to choose from multiple service providers and actively switch from one operator to another. In this highly competitive market, the telecommunications industry experiences an average of 15-25% annual churn rate. Given the fact that it costs 5-10 times more to acquire a new customer than to retain an existing one, customer retention has now become even more important than customer acquisition.


For many incumbent operators, retaining high profitable customers is the number one business goal.

To reduce customer churn, telecom companies need to predict which customers are at high risk of churn.


In this project, you will analyse customer-level data of a leading telecom firm, build predictive models to identify customers at high risk of churn and identify the main indicators of churn.


Understanding and defining churn
There are two main models of payment in the telecom industry - postpaid (customers pay a monthly/annual bill after using the services) and prepaid (customers pay/recharge with a certain amount in advance and then use the services).


In the postpaid model, when customers want to switch to another operator, they usually inform the existing operator to terminate the services, and you directly know that this is an instance of churn.


However, in the prepaid model, customers who want to switch to another network can simply stop using the services without any notice, and it is hard to know whether someone has actually churned or is simply not using the services temporarily (e.g. someone may be on a trip abroad for a month or two and then intend to resume using the services again).


Thus, churn prediction is usually more critical (and non-trivial) for prepaid customers, and the term ‘churn’ should be defined carefully.  Also, prepaid is the most common model in India and Southeast Asia, while postpaid is more common in Europe in North America.

This project is based on the Indian and Southeast Asian market.


Definitions of churn
There are various ways to define churn, such as:

Revenue-based churn: Customers who have not utilised any revenue-generating facilities such as mobile internet, outgoing calls, SMS etc. over a given period of time. One could also use aggregate metrics such as ‘customers who have generated less than INR 4 per month in total/average/median revenue’.

 The main shortcoming of this definition is that there are customers who only receive calls/SMSes from their wage-earning counterparts, i.e. they don’t generate revenue but use the services. For example, many users in rural areas only receive calls from their wage-earning siblings in urban areas.


Usage-based churn: Customers who have not done any usage, either incoming or outgoing - in terms of calls, internet etc. over a period of time.


A potential shortcoming of this definition is that when the customer has stopped using the services for a while, it may be too late to take any corrective actions to retain them. For e.g., if you define churn based on a ‘two-months zero usage’ period, predicting churn could be useless since by that time the customer would have already switched to another operator.



In this project, you will use the usage-based definition to define churn.



High-value churn
In the Indian and the Southeast Asian market, approximately 80% of revenue comes from the top 20% customers (called high-value customers). Thus, if we can reduce churn of the high-value customers, we will be able to reduce significant revenue leakage.



In this project, you will define high-value customers based on a certain metric (mentioned later below) and predict churn only on high-value customers.



Understanding the business objective and the data
The dataset contains customer-level information for a span of four consecutive months - June, July, August and September. The months are encoded as 6, 7, 8 and 9, respectively.


The business objective is to predict the churn in the last (i.e. the ninth) month using the data (features) from the first three months. To do this task well, understanding the typical customer behaviour during churn will be helpful.



Understanding customer behaviour during churn
Customers usually do not decide to switch to another competitor instantly, but rather over a period of time (this is especially applicable to high-value customers). In churn prediction, we assume that there are three phases of customer lifecycle :

The ‘good’ phase: In this phase, the customer is happy with the service and behaves as usual.

The ‘action’ phase: The customer experience starts to sore in this phase, for e.g. he/she gets a compelling offer from a  competitor, faces unjust charges, becomes unhappy with service quality etc. In this phase, the customer usually shows different behaviour than the ‘good’ months. Also, it is crucial to identify high-churn-risk customers in this phase, since some corrective actions can be taken at this point (such as matching the competitor’s offer/improving the service quality etc.)

The ‘churn’ phase: In this phase, the customer is said to have churned. You define churn based on this phase. Also, it is important to note that at the time of prediction (i.e. the action months), this data is not available to you for prediction. Thus, after tagging churn as 1/0 based on this phase, you discard all data corresponding to this phase.


In this case, since you are working over a four-month window, the first two months are the ‘good’ phase, the third month is the ‘action’ phase, while the fourth month is the ‘churn’ phase.



# 2. Importing Important Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler

In [ ]:
#Improting the PCA module
from sklearn.decomposition import PCA

In [ ]:
from sklearn.decomposition import IncrementalPCA

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

In [ ]:
from sklearn import metrics
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

In [ ]:
import statsmodels.api as sm

# 3. Data Reading and Understanding

In [ ]:
df = pd.read_csv(r'D:\BA Assignment\telecom_churn_data.csv')
df.head()

In [ ]:
df.info(verbose=True)

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.shape

In [ ]:
df.columns.tolist()

# 4. Data preparation

In [ ]:
#copy the original data
data = df.copy()

#### We will create a utility function (missing_df) which will accept the given data frame and give back a dataframe sorted in descending order of missing value percentage for each columns in the data frame passed.

In [ ]:
def missing_df(dfname):
    missing = round(100*(dfname.isnull().sum()/len(dfname.index)), 2)
    missing = pd.DataFrame(missing).reset_index()
    missing.columns = ["ColumnName","percentage"]
    missing = missing.sort_values(by=['percentage'],ascending=False)
    return(missing)

missdf = missing_df(data)
missdf.head(30)

#### If the recharge value is unavailable, it can be assumed that it has not been recharged. First, we will identify all recharge columns and impute them with zero.

In [ ]:
recharge_col = []

for col in list(data.columns):
    if (('rech' in col) and ('date' not in col)):
        recharge_col.append(col)

recharge_col

In [ ]:
data[data.total_rech_data_6.isnull() == True][['total_rech_data_6','date_of_last_rech_data_6']]

#### missing recharge value also means that , they havnt recharged
#### So will impute missing values as zero

In [ ]:
data[recharge_col].describe(include='all')

#### some shows minimun recharge as 0 and some as 1, we can assume 0 for not recharging

In [ ]:
# impute missing values with 0
data[recharge_col] = data[recharge_col].apply(lambda x: x.fillna(0))

In [ ]:
## data[recharge_col] =  data[recharge_col].apply(lambda x:0 if x is 1 else x)

In [ ]:
data[recharge_col].describe(include='all')

In [ ]:
missdf = missing_df(data)
missdf.head(40)

In [ ]:
data['last_date_of_month_7'].head()

In [ ]:
data['last_date_of_month_6'].value_counts(dropna = False)

In [ ]:
data['last_date_of_month_7'].value_counts(dropna = False)

In [ ]:
data['last_date_of_month_8'].value_counts(dropna = False)

In [ ]:
data['last_date_of_month_9'].value_counts(dropna = False)

#### we will fill missing values of last_date_of_month date with actual last dates for missing values

In [ ]:
data['last_date_of_month_7'].fillna('7/31/2014',inplace = True)
data['last_date_of_month_8'].fillna('8/31/2014',inplace = True)
data['last_date_of_month_9'].fillna('9/30/2014',inplace = True)

#### Convert all date columns to datetime format

In [ ]:
date_col = []

for col in list(data.columns):
    if ('date' in col):
        date_col.append(col)

date_col

In [ ]:
data[date_col] = data[date_col].apply(pd.to_datetime)

In [ ]:
data.info(verbose=True)

#### will create a new column wich give how many days before the laste date of month recharge was done

In [ ]:
data['duration_last_rech_6']= data['last_date_of_month_6']-data['date_of_last_rech_6']
data['duration_last_rech_7']= data['last_date_of_month_7']-data['date_of_last_rech_7']
data['duration_last_rech_8']= data['last_date_of_month_8']-data['date_of_last_rech_8']
data['duration_last_rech_9']= data['last_date_of_month_9']-data['date_of_last_rech_9']

In [ ]:
data['duration_last_rech_6']=data['duration_last_rech_6'].astype(str)
data['duration_last_rech_6']=data['duration_last_rech_6'].str.split(" ",n=3,expand=True)[0]
data.loc[(data['duration_last_rech_6']=='NaT'),'duration_last_rech_6']=-1
data['duration_last_rech_6']=data['duration_last_rech_6'].astype(int)

In [ ]:
data['duration_last_rech_7']=data['duration_last_rech_7'].astype(str)
data['duration_last_rech_7']=data['duration_last_rech_7'].str.split(" ",n=3,expand=True)[0]
data.loc[(data['duration_last_rech_7']=='NaT'),'duration_last_rech_7']= -1
data['duration_last_rech_7']=data['duration_last_rech_7'].astype(int)

In [ ]:
data['duration_last_rech_8']=data['duration_last_rech_8'].astype(str)
data['duration_last_rech_8']=data['duration_last_rech_8'].str.split(" ",n=3,expand=True)[0]
data.loc[(data['duration_last_rech_8']=='NaT'),'duration_last_rech_8']= -1
data['duration_last_rech_8']=data['duration_last_rech_8'].astype(int)

In [ ]:
data['duration_last_rech_9']=data['duration_last_rech_9'].astype(str)
data['duration_last_rech_9']=data['duration_last_rech_9'].str.split(" ",n=3,expand=True)[0]
data.loc[(data['duration_last_rech_9']=='NaT'),'duration_last_rech_9']= -1
data['duration_last_rech_9']=data['duration_last_rech_9'].astype(int)

In [ ]:
data['duration_last_rech_data_6']= data['last_date_of_month_6']-data['date_of_last_rech_data_6']
data['duration_last_rech_data_7']= data['last_date_of_month_7']-data['date_of_last_rech_data_7']
data['duration_last_rech_data_8']= data['last_date_of_month_8']-data['date_of_last_rech_data_8']
data['duration_last_rech_data_9']= data['last_date_of_month_9']-data['date_of_last_rech_data_9']

In [ ]:
data['duration_last_rech_data_6']=data['duration_last_rech_data_6'].astype(str)
data['duration_last_rech_data_6']=data['duration_last_rech_data_6'].str.split(" ",n=3,expand=True)[0]
data.loc[(data['duration_last_rech_data_6']=='NaT'),'duration_last_rech_data_6']= -1
data['duration_last_rech_data_6']=data['duration_last_rech_data_6'].astype(int)

In [ ]:
data['duration_last_rech_data_7']=data['duration_last_rech_data_7'].astype(str)
data['duration_last_rech_data_7']=data['duration_last_rech_data_7'].str.split(" ",n=3,expand=True)[0]
data.loc[(data['duration_last_rech_data_7']=='NaT'),'duration_last_rech_data_7']= -1
data['duration_last_rech_data_7']=data['duration_last_rech_data_7'].astype(int)

In [ ]:
data['duration_last_rech_data_8']=data['duration_last_rech_data_8'].astype(str)
data['duration_last_rech_data_8']=data['duration_last_rech_data_8'].str.split(" ",n=3,expand=True)[0]
data.loc[(data['duration_last_rech_data_8']=='NaT'),'duration_last_rech_data_8']= -1
data['duration_last_rech_data_8']=data['duration_last_rech_data_8'].astype(int)

In [ ]:
data['duration_last_rech_data_9']=data['duration_last_rech_data_9'].astype(str)
data['duration_last_rech_data_9']=data['duration_last_rech_data_9'].str.split(" ",n=3,expand=True)[0]
data.loc[(data['duration_last_rech_data_9']=='NaT'),'duration_last_rech_data_9']= -1
data['duration_last_rech_data_9']=data['duration_last_rech_data_9'].astype(int)

#### Now we can drop the date columns as new meaningful columns are derrived

In [ ]:
data.drop(date_col,axis=1,inplace=True)
data.shape

In [ ]:
missdf = missing_df(data)
missdf.set_index('ColumnName',inplace=True)
missdf.head(41)

In [ ]:
miss_col = missdf[missdf.percentage >0].index.tolist()
miss_col

#### Group MOU(Minutes of Usage ) and OTH (oter mobiles) columns to two lists

In [ ]:
cols_mou = []
cols_other = []
for col in miss_col:
    if 'mou' in col:
        cols_mou.append(col)
    if 'others' in col:
        cols_other.append(col)

print(len(cols_mou), len(cols_other))


In [ ]:
cols_other

In [ ]:
data['ic_others_6'].value_counts(dropna = False)

In [ ]:
data['og_others_6'].value_counts(dropna = False)

In [ ]:
data[data['og_others_6'].isnull() == True]['duration_last_rech_6'].value_counts(dropna=False)

#### we will inpute with 0 for all the missing columns as its the mode

In [ ]:
data[cols_mou].mode()

In [ ]:
data[cols_other].mode()

In [ ]:
for col in cols_mou:
    data[col].fillna(0,inplace = True)

for col in cols_other:
    data[col].fillna(0,inplace = True)


In [ ]:
missdf = missing_df(data)
missdf.set_index('ColumnName',inplace=True)
missdf.head(20)

In [ ]:
data.info()

In [ ]:
data['night_pck_user_6'].value_counts(dropna=False)

In [ ]:
#### we will use

In [ ]:
# Fetching all categorical columns

col_categorical=['night_pck_user_6','night_pck_user_7','night_pck_user_8','night_pck_user_9','fb_user_6','fb_user_7','fb_user_8','fb_user_9']

#### for categorical columns , missing value will set as new categorical value -1

In [ ]:
# Imputing missing categories as -1

data[col_categorical]=data[col_categorical].apply(lambda x: x.fillna(-1))

In [ ]:
missdf = missing_df(data)
missdf.set_index('ColumnName',inplace=True)
missdf.head(20)

#### Average revenue per user, if missing will set to 0

In [ ]:
arpu_col = []
for col in missdf.index.tolist():
    if 'arpu' in col and data[col].isnull().sum() > 0:
        arpu_col.append(col)

arpu_col

In [ ]:
# Imputing zeroes for all recharge columns

data[arpu_col]=data[arpu_col].apply(lambda x: x.fillna(0))

In [ ]:
missdf = missing_df(data)
missdf.set_index('ColumnName',inplace=True)
missdf.head(20)

# 5. Filter High Value customers

#### High-value churn
#### In the Indian and the Southeast Asian market, approximately 80% of revenue comes from the top 20% customers (called high-value customers). Thus, if we can reduce churn of the high-value customers, we will be able to reduce significant revenue leakage.

#### In this project, you will define high-value customers based on a certain metric (mentioned later below) and predict churn only on high-value customers.

#### Good Phase (June & July)
1. Calculate average recharge done by customer in June and July(total_rech_amt)
2. Look at the 70th percentile recharge amount
3. Retain only those customers who have recharged their mobiles with more than or equal to 70th percentile amount

#### Find the coulmns having recharge related info

In [ ]:
recharge_col

In [ ]:
data[['av_rech_amt_data_6','count_rech_2g_6','count_rech_3g_6', 'total_rech_data_6','total_rech_amt_6','total_rech_num_6']]

#### Step 1: We need to find the average recharge done by customer in the good phase
####   For that, we will create a new column total recharge amount for data: ''total_rech_amt_data"  , which can be calculated by multiplying the average recharge data with count of  data recharge.

In [ ]:
data['total_rech_amt_data_6'] = data.av_rech_amt_data_6 * data.total_rech_data_6
data['total_rech_amt_data_7'] = data.av_rech_amt_data_7 * data.total_rech_data_7
data['total_rech_amt_data_8'] = data.av_rech_amt_data_8 * data.total_rech_data_8

####  Now to Calculate average recharge done by customer in June and July(total_rech_amt) months , we will calculate total recharge amount for  june and july months by adding total data recharge and call recharge for two months(june & july) and average it.

In [ ]:
data['total_avg_rech_amnt_good_phase'] = (data.total_rech_amt_6 + data.total_rech_amt_data_6 \
                                               + data.total_rech_amt_7+ data.total_rech_amt_data_7)/2

#### Step 2: Look at 70th percentile of total_avg_rech_amnt_good_phase

In [ ]:
high_value_filter = data.total_avg_rech_amnt_good_phase.quantile(0.7)
high_value_filter

#### Step3: Retain only those customers who have recargted teir mobile more than or equal to 70th percentile amount

In [ ]:
data_hvc = data[data.total_avg_rech_amnt_good_phase > high_value_filter]
data_hvc.head()

In [ ]:
data_hvc.shape

# 6. Derrive Churn ( Target Variable)

#### 9th Month is our Churn Phase. Usage-based churn
#### we need to follow below steps to find the chhurn
1. Calculate total incoming and outgoing minutes of usage
2. Calculate 2g and 3g data consumption
3. Create churn variable: those who have not used either calls or internet in the month of September are customers who have churned (churn =1, else 0)
4. Check Churn percentage.
5. Delete columns that belong to the churn month

#### First we will find the columns to be used for calculating churn . These will be columns which gives total incoming and outgoing minutes of usage and 2g & 3g data consumption

In [ ]:
churn_var_9 = []
for col in data_hvc.columns.tolist():
    if '_9' in col:
        if 'ic_mou' or 'og_mou' or '_2g_' or '_3g_' in col:
            churn_var_9.append(col)

churn_var_9

####  'vol_2g_mb_9', 'vol_3g_mb_9','total_og_mou_9','total_ic_mou_9' => these columns can be used for calculating churn

In [ ]:
hvc_9 = ['vol_2g_mb_9', 'vol_3g_mb_9','total_og_mou_9','total_ic_mou_9' ]

In [ ]:
df = data_hvc[hvc_9].reset_index(drop  = True)
df.head()

In [ ]:
missdf = missing_df(df)
missdf.head()

####  step3: Create Churn Variable

In [ ]:
# Initially set all the values as 0
data_hvc['churn']= 0

In [ ]:
is_churned = (data_hvc.total_ic_mou_9 == 0) & \
             (data_hvc.total_og_mou_9 == 0) & \
             (data_hvc.vol_2g_mb_9 ==0) & \
             (data_hvc.vol_3g_mb_9 ==0)
# set all which having is_churned True condition as 1
data_hvc.loc[is_churned,'churn']=1

#### Check the churn percentage

In [ ]:
# let us check what's the % of churned customers
100*data_hvc.churn.sum()/len(data_hvc)

#### From  this it is evident that the dataset is highly imbalanced. The proportion for churn to non-churn is around 8%. For a correct and smooth analysis we need to deal with this class imbalance problem. We will deal with this in a later section.

#### Delete the columns tat belongs to the churn month

In [ ]:
churn_col =  data_hvc.columns[data_hvc.columns.str.contains('_9')]
churn_col

In [ ]:
# drop all columns corresponding to the churn phase
data_hvc.drop(churn_col,axis=1,inplace=True)

# 7. Data Preparation & EDA

In [ ]:
data_hvc.shape

In [ ]:
data_hvc.head()

#### We can drop columns having unique identifiers and single values

In [ ]:
unique_col = ['mobile_number','circle_id']


#### We will remove the columns aving only single value for all rows

In [ ]:
for col in data_hvc.columns.tolist():
    if (len(data_hvc[col].unique().tolist()) ==1):
        unique_col.append(col)

unique_col

In [ ]:
data_hvc.drop(unique_col,axis=1,inplace = True)

In [ ]:
data_hvc.reset_index(drop=True,inplace=True)

In [ ]:
data_hvc.head()

In [ ]:
data_hvc.info()

In [ ]:
cat_col = []
for col in data_hvc.columns.tolist():
    if (len(data_hvc[col].unique().tolist()) < 17):
        cat_col.append(col)
        print("******************************* \n")
        print(col)
        print(data_hvc[col].unique().tolist())
        print("******************************* \n")

In [ ]:
print(len(cat_col) , cat_col)

In [ ]:
int_col = data_hvc.columns.tolist()
for col in cat_col:
    if col in int_col:
        int_col.remove(col)

int_col

In [ ]:
int_col


In [ ]:
col_month_6 = []
col_month_7 = []
col_month_8 = []
col_non_month = []

for col in int_col:
    if '_6' in col:
        col_month_6.append(col)
    elif '_7' in col:
        col_month_7.append(col)
    elif '_8' in col:
        col_month_8.append(col)
    else:
        col_non_month.append(col)

print("col_month_6, length ",len(col_month_6),col_month_6)
print("\n")
print("col_month_7, length ", len(col_month_7),col_month_7)
print("\n")
print("col_month_8, length ",len(col_month_8), col_month_8)
print("\n")
print("col_non_month, length ",len(col_non_month), col_non_month)



#### We will define some utility function below wich will help us to plot the graphs

In [ ]:
# Plot box plot for all columns in the list input_cols.
def box_plot(data,input_cols,ncol):
    leng = len(input_cols)
    if leng%ncol == 0:
        rows = leng//ncol
    else:
        rows = leng//ncol + 1

    figure, axes = plt.subplots(nrows=rows, ncols=ncol,figsize=(20,3.5*rows))

    for i, xvar in enumerate(input_cols):
            axes[i//ncol,i%ncol].title.set_text(xvar)
            axes[i//ncol,i%ncol].tick_params(axis='x', rotation=45)

            axes[i//ncol,i%ncol].boxplot(data[xvar])



    figure.tight_layout(pad=3.0)
    plt.show()


# Plot histogram for all columns in the list input_cols.
def plot_histogram(data, input_cols,ncol):
    leng = len(input_cols)
    if leng%ncol == 0:
        rows = leng//ncol
    else:
        rows = leng//ncol + 1
    fig = plt.figure(figsize=(20,3.5*rows))

    for i, xvar in enumerate(input_cols):

            fig.add_subplot(rows,ncol,i+1).tick_params(axis='x', rotation=45)
            fig.add_subplot(rows,ncol,i+1).title.set_text(xvar + " histogram")
            sns.distplot(data[xvar],hist = True)

    fig.tight_layout(pad=3.0)


# create box plot for  6th, 7th and 8th month
def plot_box_chart(attribute):
    plt.figure(figsize=(20,16))
    df = data_hvc
    plt.subplot(2,3,1)
    sns.boxplot(data=df, y=attribute+"_6",x="churn",hue="churn",
                showfliers=False,palette=("magma"))
    plt.subplot(2,3,2)
    sns.boxplot(data=df, y=attribute+"_7",x="churn",hue="churn",
                showfliers=False,palette=("magma"))
    plt.subplot(2,3,3)
    sns.boxplot(data=df, y=attribute+"_8",x="churn",hue="churn",
                showfliers=False,palette=("magma"))
    plt.show()


# Custom Function for Default Plotting variables

# Function Parameters  -

# figure_title         -    The title to use for the plot.
# xlabel               -    The x-axis label for the plot.
# ylabel               -    The y-axis label for the plot.

def set_plotting_variable(figure_title, xlabel, ylabel):

    plt.title(figure_title)
    plt.xlabel(xlabel, labelpad = 15)
    plt.ylabel(ylabel, labelpad = 10)



# Custom Function to add data labels in the graph

def add_data_labels(ax, spacing = 5):

    # For each bar: Place a label
    for rect in ax.patches:
        # Get X and Y placement of label from rect.
        y_value = rect.get_height()
        x_value = rect.get_x() + rect.get_width() / 2

        # Number of points between bar and label. Change to your liking.
        space = spacing
        # Vertical alignment for positive values
        va = 'bottom'

        # If value of bar is negative: Place label below bar
        if y_value < 0:
            # Invert space to place label below
            space *= -1
            # Vertically align label at top
            va = 'top'

        # Use Y value as label and format number with one decimal place
        label = "{:.2f}%".format(y_value)

        # Create annotation
        plt.annotate(
            label,                        # Use `label` as label
            (x_value, y_value),           # Place label at end of the bar
            xytext = (0, space),          # Vertically shift label by `space`
            textcoords = "offset points", # Interpret `xytext` as offset in points
            ha = 'center',                # Horizontally center label
            va = va)                      # Vertically align label differently for positive and negative values.




# Custom Function for Univariate Analysis

# Function Parameters   -

# figsize_x             -      The width of the plot figure in inches.
# figsize_y             -      The height of the plot figure in inches.
# subplot_x             -      The rows for the subplot.
# subplot_y             -      The columns for the subplot.
# xlabel                -      The x-axis label for the plot.
# ylabel                -      The y-axis label for the plot.
# x_axis                -      The series/variable to be plotted along the x-axis.
# data                  -      The data frame.

# wspace                -      The amount of width reserved for space between subplots,
#                              expressed as a fraction of the average axis width

# xlabel_rotation       -      The degree of rotation for the x-axis ticks (values).

def plot_univariate(figsize_x, figsize_y, subplot_x, subplot_y, xlabel, ylabel, x_axis, data, wspace):

    plt.figure(figsize = (figsize_x, figsize_y))

    title_1 = "Distribution Plot of " + xlabel
    title_2 = "Box Plot of " + xlabel

    # Subplot - 1
    plt.subplot(subplot_x, subplot_y, 1)

    sns.distplot(data[x_axis], hist = True, kde = True, color = 'g')
    # Call Custom Function
    set_plotting_variable(title_1, xlabel, ylabel)

    # Subplot - 2
    plt.subplot(subplot_x, subplot_y, 2)

    sns.boxplot(x = x_axis, data = data, color = 'm')
    # Call Custom Function
    set_plotting_variable(title_2, xlabel, ylabel)

    plt.subplots_adjust(wspace = wspace)
    plt.show()

# Custom Function for Bivariate Analysis

# Function Parameters   -

# y_axis                -      The series/variable to be plotted along the y-axis.

def plot_bivariate(y_axis):

    plt.figure(figsize = (15, 5))

    xlabel = "Churn"
    x_axis = "churn"

    title_1 = "Month 6 - " + xlabel
    title_2 = "Month 7 - " + xlabel
    title_3 = "Month 8 - " + xlabel

    print("\nData Visualization of churn vs " + y_axis)

    # Subplot - 1
    plt.subplot(1, 3, 1)

    sns.boxplot(x = x_axis, y = y_axis + "_6", hue = "churn", data = data_hvc, showfliers = False)
    #sns.barplot(x = x_axis, y = y_axis + "_6", hue = "churn", data = data_hvc)
    # Call Custom Function
    set_plotting_variable(title_1, xlabel, y_axis + "_6")

    # Subplot - 2
    plt.subplot(1, 3, 2)

    sns.boxplot(x = x_axis, y = y_axis + "_7", hue = "churn", data = data_hvc, showfliers = False)
    #sns.barplot(x = x_axis, y = y_axis + "_7", hue = "churn", data = data_hvc)
    # Call Custom Function
    set_plotting_variable(title_2, xlabel, y_axis + "_7")

    # Subplot - 3
    plt.subplot(1, 3, 3)

    sns.boxplot(x = x_axis, y = y_axis + "_8", hue = "churn", data = data_hvc, showfliers = False)
    #sns.barplot(x = x_axis, y = y_axis + "_8", hue = "churn", data = data_hvc)
    # Call Custom Function
    set_plotting_variable(title_3, xlabel, y_axis + "_8")

    plt.subplots_adjust(wspace = 0.4)
    plt.show()


#### Univariate Plot Analysis of Quantitative Variables

#### Boxplot non Monthly columns

In [ ]:
# INT Data
a = 3  # number of rows
b = 2 # number of columns
c = 1  # initialize plot counter

fig = plt.figure(figsize=(12,12))

for i in col_non_month:
    plt.subplot(a, b, c)
    plt.title('{}, subplot: {}{}{}'.format(i, a, b, c))
    plt.xlabel(i)
    sns.boxplot(data_hvc[i])
    c = c + 1

fig.tight_layout()
plt.show()


#### Observation

 There are outliers

In [ ]:

counter = 1

for col_list in data_hvc.columns:

    if col_list not in cat_col:

        # Call Custom Function
        plot_univariate(figsize_x = 20,
                        figsize_y = 8,
                        subplot_x = 1,
                        subplot_y = 2,
                        xlabel = col_list,
                        ylabel = "Distribution",
                        x_axis = col_list,
                        data = data_hvc,
                        wspace = 0.2)

        counter += 1

#### There are lot of Outliers present in the variables. We will remove these outliers

#### some of the arpu (average revenue per user) value is less than zero
#### Now the revenue generated from a user cannot be negative. If a customer is not using any services then apru for the person would be zero (rather that being negative). Now if arpu is negative for any row, then that would mean that is a wrong/corrupt data. It will be of no use to us for analysis. We will drop such observations.

In [ ]:
# Index where the arpu values for month 6 are less than 0 -

arpu_6_index = (data_hvc['arpu_6'] < 0)

# Total number of such observations for month 6 -
print('Total observations with negative arpu values for month 6 -', arpu_6_index.sum())

In [ ]:
# Index where the arpu values for month 7 are less than 0 -

arpu_7_index = (data_hvc['arpu_7'] < 0)

# Total number of such observations for month 6 -
print('Total observations with negative arpu values for month 7 -', arpu_7_index.sum())

In [ ]:
# Index where the arpu values for month 8 are less than 0 -

arpu_8_index = (data_hvc['arpu_8'] < 0)

# Total number of such observations for month 6 -
print('Total observations with negative arpu values for month 8 -', arpu_8_index.sum())

#### we will delete these wrong observations

In [ ]:
# Let's delete the observations with negative arpu values.

data_hvc = data_hvc[(data_hvc['arpu_6'] >= 0) & (data_hvc['arpu_7'] >= 0) & (data_hvc['arpu_8'] >= 0)]

#### Now we will remove te outliers

#### We will Cap outliers in all numeric variables with k-sigma technique

In [ ]:
def cap_outliers(array, k=3):
    upper_limit = array.mean() + k*array.std()
    lower_limit = array.mean() - k*array.std()
    array[array<lower_limit] = lower_limit
    array[array>upper_limit] = upper_limit
    return array

In [ ]:

# cap outliers in the numeric columns
data_hvc[int_col] = data_hvc[int_col].apply(cap_outliers, axis=0)
data_hvc.head()

#### we will plot again and confirm now no outliers

In [ ]:

counter = 1

for col_list in data_hvc.columns:

    if col_list not in cat_col:

        # Call Custom Function
        plot_univariate(figsize_x = 20,
                        figsize_y = 8,
                        subplot_x = 1,
                        subplot_y = 2,
                        xlabel = col_list,
                        ylabel = "Distribution",
                        x_axis = col_list,
                        data = data_hvc,
                        wspace = 0.2)

        counter += 1

In [ ]:
plot_histogram(data_hvc,col_non_month,2)

#### Bivariate for category columns

In [ ]:
for col in cat_col:
    data_hvc.groupby(col)['total_avg_rech_amnt_good_phase'].sum().plot.barh()
    plt.show()

#### Observation :

- people who haven't selected night_pck,  monthly_2g,monthly_3g  are spending more average amount
- People selected fb_user are spending more average amount


####  Bivariate Plot Analysis of Ordered categorical variables vs Percentage Rate

#### we will plot each using boxplot

In [ ]:
for i in cat_col:
    fig = plt.figure(figsize=(15,11))
    sns.boxplot(data=data_hvc,x=i,y="total_avg_rech_amnt_good_phase")
    fig.tight_layout()
    plt.show()

In [ ]:
cat_col.remove('churn')

In [ ]:
counter = 1

plt.figure(figsize = (15, 12))

for col_list in cat_col:

    series = round(((data_hvc[col_list].value_counts(dropna = False))/(len(data_hvc[col_list])) * 100), 2)

    plt.subplot(4, 4, counter)
    ax = sns.barplot(x = series.index, y = series.values, order = series.sort_index().index)
    plt.xlabel(col_list, labelpad = 15)
    plt.ylabel('Percentage Rate', labelpad = 10)

    # Call Custom Function
    add_data_labels(ax)

    counter += 1

del counter, ax

plt.subplots_adjust(hspace = 0.3)
plt.subplots_adjust(wspace = 0.5)
plt.show()

#### Bivariate Analysis

In [ ]:
# Bivariate Analysis

plot_bivariate("arpu")

plot_bivariate("onnet_mou")

plot_bivariate("offnet_mou")

plot_bivariate("total_og_mou")

plot_bivariate("total_ic_mou")

plot_bivariate("total_rech_num")

plot_bivariate("total_rech_amt")

plot_bivariate("total_rech_data")

plot_bivariate("vol_2g_mb")

plot_bivariate("vol_3g_mb")

plot_bivariate("total_rech_amt_data")

#### here is a significant drop in the columns for data in 8th month for churned customers.

####  Looking at the problem statement, attributes total_ic_mou_6, total_og_mou_6, vol_2g_mb_ 6and vol_3g_mb_ 6are used to tag churn. So it is evident from the problem statement that the individual incoming and outgoing attributes are not used for data analysis. Dropping the individual columns (whose totals are already available like incoming, outgoing, arpu, etc) can help us in better analysis. Also, dropping these individual columns will help in removing the multicollinearity.

#### Let's now drop all those individual columns whose totals are available.

In [ ]:
# Let's drop individual columns whose totals are available as a different attribute

single_cols = ['loc_ic_t2t_mou_6', 'loc_ic_t2t_mou_7', 'loc_ic_t2t_mou_8',
                   'loc_ic_t2m_mou_6', 'loc_ic_t2m_mou_7', 'loc_ic_t2m_mou_8',
                   'loc_ic_t2f_mou_6', 'loc_ic_t2f_mou_7', 'loc_ic_t2f_mou_8',
                   'std_ic_t2t_mou_6', 'std_ic_t2t_mou_7', 'std_ic_t2t_mou_8',
                   'std_ic_t2m_mou_6', 'std_ic_t2m_mou_7', 'std_ic_t2m_mou_8',
                   'std_ic_t2f_mou_6', 'std_ic_t2f_mou_7', 'std_ic_t2f_mou_8',
                   'loc_og_t2t_mou_6', 'loc_og_t2t_mou_7', 'loc_og_t2t_mou_8',
                   'loc_og_t2m_mou_6', 'loc_og_t2m_mou_7', 'loc_og_t2m_mou_8',
                   'loc_og_t2f_mou_6', 'loc_og_t2f_mou_7', 'loc_og_t2f_mou_8',
                   'loc_og_t2c_mou_6', 'loc_og_t2c_mou_7', 'loc_og_t2c_mou_8',
                   'std_og_t2t_mou_6', 'std_og_t2t_mou_7', 'std_og_t2t_mou_8',
                   'std_og_t2m_mou_6', 'std_og_t2m_mou_7', 'std_og_t2m_mou_8',
                   'std_og_t2f_mou_6', 'std_og_t2f_mou_7', 'std_og_t2f_mou_8',
                   'last_day_rch_amt_6', 'last_day_rch_amt_7', 'last_day_rch_amt_8',
                   'arpu_3g_6', 'arpu_3g_7', 'arpu_3g_8',
                   'arpu_2g_6', 'arpu_2g_7', 'arpu_2g_8',
                   'av_rech_amt_data_6', 'av_rech_amt_data_7', 'av_rech_amt_data_8']

data_hvc[single_cols]

In [ ]:
data_hvc.drop(single_cols, axis = 1, inplace = True)

data_hvc.shape

In [ ]:
corr_matrix = data_hvc.corr().abs()

#the matrix is symmetric so we need to extract upper triangle matrix without diagonal (k = 1)
upper_triangle = (corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool)))

highly_correlated_features = [column for column in upper_triangle.columns if any(upper_triangle[column] > 0.80)]
print("List of highly correlated features from the above plot - \n\n", highly_correlated_features)
print("\n\nTotal features with high correlation - ", len(highly_correlated_features))

In [ ]:
data_hvc.shape

In [ ]:
data_hvc.reset_index(drop=True)
data_hvc.head()

In [ ]:
fig,ax = plt.subplots(figsize=(11,9))
sns.heatmap(data_hvc[cat_col].corr(),annot=True,cmap="Blues")
plt.show()

#### night_pack_user & fb user are highly correlated. So we can drop one

In [ ]:
cat_col

In [ ]:
data_hvc.drop(['night_pck_user_6','night_pck_user_7','night_pck_user_8'],axis=1,inplace=True)
data_hvc.shape

In [ ]:
mou_col = []
for col in data_hvc.columns.tolist():
    if 'mou' in col and col:
        mou_col.append(col)

len(mou_col)


In [ ]:
fig,ax = plt.subplots(figsize=(20,11))
sns.heatmap(data_hvc[mou_col].corr(),annot=True,cmap="Blues")
plt.show()

#### high correlation bwn -> total_ic_mou_6' & loc_ic_mou_6, total_ic_mou_7' & loc_ic_mou_7, total_ic_mou_8' & loc_ic_mou_8


In [ ]:
data_hvc.drop(['loc_ic_mou_6','loc_ic_mou_7','loc_ic_mou_8'],axis=1,inplace=True)

data_hvc.shape

#### check correlation

In [ ]:
data_hvc.corr()

In [ ]:
corr_matrix = data_hvc.corr().abs()

#the matrix is symmetric so we need to extract upper triangle matrix without diagonal (k = 1)
upper_triangle = (corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool)))

highly_correlated_features = [column for column in upper_triangle.columns if any(upper_triangle[column] > 0.80)]
print("List of highly correlated features from the above plot - \n\n", highly_correlated_features)
print("\n\nTotal features with high correlation - ", len(highly_correlated_features))

#### Number of highly correlated features have reduced. Now we will leave this high correlation as it is  since we will use PCA and other techniques to remove it in later part

### Deriving new features

#### Understanding customer behaviour during churn
##### The churn prediction model assumes that customers switch to competitors over time, particularly high-value ones, and involves three phases of the customer lifecycle.

##### The ‘good’ phase: In this phase, the customer is happy with the service and behaves as usual.

##### During the 'action' phase, customers experience discomfort, may face competitor offers, unjust charges, or poor service quality, necessitating corrective actions to improve their experience.

##### The 'churn' phase refers to a customer's behavior during which data is not available for prediction, and after tagging it as 1/0, all data is discarded.

 ##### In this case, since you are working over a four-month window, the first two months are the ‘good’ phase, the third month is the ‘action’ phase, while the fourth month is the ‘churn’ phase.

#### So we will create some new fields by combining  some of the data for good phase and taking average of it , also by finding the difference from good phase and action phase

In [ ]:
# we will define a Custom Function to derive new good phase columns (combining 6 & 7 months) and drop the original columns
def derive_good_action_phase(df, col):

    col_6 = col + "_6"
    col_7 = col + "_7"
    col_8 = col + "_8"
    good_phase_col = col + "_good_phase"
    action_phase_col = col + "_action_phase"

    df[good_phase_col] = (df[col_6] + df[col_7])/2
    df[action_phase_col] = df[col_8] - df[good_phase_col]

    df.drop([col_6, col_7, col_8], axis = 1, inplace = True)

    return df

#### Copying to final_data just to restore in case of any issues

In [ ]:
final_data = data_hvc.copy()

In [ ]:
feature_list = ['arpu',"onnet_mou","offnet_mou","roam_ic_mou","roam_og_mou","loc_og_mou","std_og_mou","isd_og_mou","spl_og_mou",
                "og_others","total_og_mou","std_ic_mou","spl_ic_mou","isd_ic_mou","ic_others","total_ic_mou",
               "total_rech_num","total_rech_amt","max_rech_amt","total_rech_data","max_rech_data","count_rech_2g","count_rech_3g",
               "vol_2g_mb","vol_3g_mb","monthly_2g","sachet_2g","monthly_3g","sachet_3g","total_rech_amt_data"]

for col in feature_list:
    data_hvc = derive_good_action_phase(data_hvc, col)

data_hvc.head()


In [ ]:
data_hvc.shape

In [ ]:

data_hvc.head()

In [ ]:
len(data_hvc.columns)

# 8 .Modeling

In [ ]:
# Separate input features and target
y = data_hvc.churn
X = data_hvc.drop('churn', axis=1)

In [ ]:
cols = X.columns
X = pd.DataFrame(scale(X))
X.columns = cols
X.columns

In [ ]:
# setting up testing and training sets
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, test_size=0.3, random_state=100)

### Handling Class Imbalance using SMOTE

In [ ]:
data_hvc['churn'].value_counts().plot(kind= 'bar').set_title('churned')

In [ ]:

smote  =  SMOTE()
X_train, y_train = smote.fit_resample(X_train, y_train)

#### we will check if data looks balanced now

In [ ]:
pd.DataFrame(y_train).churn.value_counts().plot(kind= 'bar').set_title('churned')

####  now both churned and non churned data is balanced.

### Performing PCA for feature reduction

In [ ]:
X_train.shape

In [ ]:
pca =  PCA(random_state=42)

In [ ]:
pca.fit(X_train)

In [ ]:
pca.components_

In [ ]:
pca.explained_variance_ratio_

In [ ]:
var_cumu = np.cumsum(pca.explained_variance_ratio_)

In [ ]:
fig = plt.figure(figsize=[12,8])
plt.vlines(x=15, ymax=1, ymin=0, colors="r", linestyles="--")
plt.hlines(y=0.95, xmax=70, xmin=0, colors="g", linestyles="--")
plt.plot(var_cumu)
plt.ylabel("Cumulative variance explained")
plt.show()

In [ ]:
### As per this Using 16 Variable can explain 90% of the variance

In [ ]:
pca_final = IncrementalPCA(n_components=16)

In [ ]:
df_train_pca = pca_final.fit_transform(X_train)

In [ ]:
df_train_pca.shape

In [ ]:
corrmat = np.corrcoef(df_train_pca.transpose())

In [ ]:
corrmat.shape

In [ ]:
plt.figure(figsize=[15,15])
sns.heatmap(corrmat, annot=True)

In [ ]:
df_test_pca = pca_final.transform(X_test)
df_test_pca.shape

### Applying logistic regression on the data on our Principal components

In [ ]:
learner_pca = LogisticRegression()

In [ ]:
model_pca = learner_pca.fit(df_train_pca, y_train)

In [ ]:
pred_probs_test = model_pca.predict_proba(df_test_pca)

In [ ]:
y_train_pred = model_pca.predict_proba(df_train_pca)[:,1]
y_train_pred

In [ ]:
y_train_pred_final = pd.DataFrame({'Churn':y_train, 'Churn_Prob':y_train_pred})
y_train_pred_final.head()

In [ ]:

# Let's create columns with different probability cutoffs
numbers = [float(x)/10 for x in range(10)]

for i in numbers:
    y_train_pred_final[i]= y_train_pred_final.Churn_Prob.map(lambda x: 1 if x > i else 0)
y_train_pred_final.head()

In [ ]:
### Now let's calculate accuracy sensitivity and specificity for various probability cutoffs.
cutoff_df = pd.DataFrame( columns = ['prob','accuracy','sensi','speci'])

# TP = confusion[1,1] # true positive
# TN = confusion[0,0] # true negatives
# FP = confusion[0,1] # false positives
# FN = confusion[1,0] # false negatives

num = [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
for i in num:
    cm1 = confusion_matrix(y_train_pred_final.Churn, y_train_pred_final[i] )
    total1=sum(sum(cm1))
    accuracy = (cm1[0,0]+cm1[1,1])/total1

    speci = cm1[0,0]/(cm1[0,0]+cm1[0,1])
    sensi = cm1[1,1]/(cm1[1,0]+cm1[1,1])
    cutoff_df.loc[i] =[ i ,accuracy,sensi,speci]
print(cutoff_df)

In [ ]:
# Let's plot accuracy sensitivity and specificity for various probabilities.
cutoff_df.plot.line(x='prob', y=['accuracy','sensi','speci'])
plt.show()

In [ ]:
### From the curve above, 0.53 is the optimum point to take it as a cutoff probability.

In [ ]:
y_train_pred_final['final_predicted'] = y_train_pred_final.Churn_Prob.map( lambda x: 1 if x > 0.5 else 0)

y_train_pred_final.head()

In [ ]:
confusion = metrics.confusion_matrix(y_train_pred_final.Churn, y_train_pred_final.final_predicted )
confusion

In [ ]:
#Making prediction on the test data
pred_probs_test = model_pca.predict_proba(df_test_pca)[:,1]
y_test_df=pd.DataFrame(y_test)
y_pred_df=pd.DataFrame(pred_probs_test)
y_test_df.reset_index(drop=True, inplace=True)
y_pred_df.reset_index(drop=True, inplace=True)
y_test_pred_final=pd.concat([y_test_df, y_pred_df],axis=1)

In [ ]:
"{:2.2}".format(metrics.roc_auc_score(y_test, pred_probs_test))



In [ ]:
# Renaming the column
y_test_pred_final= y_test_pred_final.rename(columns={ 0 : 'Churn_prob'})

In [ ]:
y_test_pred_final.head()

In [ ]:
y_test_pred_final['final_predicted'] = y_test_pred_final.Churn_prob.map(lambda x: 1 if x > 0.5 else 0)

In [ ]:
y_test_pred_final.info()

In [ ]:
#y_test_pred_final['churn']=pd.to_numeric(y_test_pred_final['churn'])
#y_test_pred_final.info()
confusion2 = metrics.confusion_matrix(y_test_pred_final.churn, y_test_pred_final.final_predicted )

In [ ]:
confusion2

In [ ]:
TP = confusion2[1,1] # true positive
TN = confusion2[0,0] # true negatives
FP = confusion2[0,1] # false positives
FN = confusion2[1,0] # false negatives

In [ ]:
# Let's see the sensitivity of our logistic regression model
TP / float(TP+FN)

In [ ]:
# Positive predictive value
print (TP / float(TP+FP))

In [ ]:
# Negative predictive value
print (TN / float(TN+ FN))

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve( y_test, pred_probs_test, drop_intermediate = False )

In [ ]:
#ROC curve code snippet from external source(Module notes)
def draw_roc( actual, probs ):
    fpr, tpr, thresholds = metrics.roc_curve( actual, probs,
                                              drop_intermediate = False )
    auc_score = metrics.roc_auc_score( actual, probs )
    plt.figure(figsize=(5, 5))
    plt.plot( fpr, tpr, label='ROC curve (area = %0.2f)' % auc_score )
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate or [1 - True Negative Rate]')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()

    return None

In [ ]:
draw_roc(y_test, pred_probs_test)

## Using Random Forest Model

In [ ]:
# Running the random forest with default parameters.
rfc = RandomForestClassifier()
rfc.fit(df_train_pca,y_train)

In [ ]:
# Making predictions
predictions = rfc.predict(df_test_pca)

In [ ]:
print(classification_report(y_test,predictions))

####  GridSearchCV to find optimal min_samples_split

In [ ]:

# Create the parameter grid based on the results of random search
param = {
    'max_depth': [1, 2, 5, 10, 20],
    'min_samples_leaf': [5, 10, 20, 50, 100],
    'max_features': [2,3,4],
    'n_estimators': [10, 30, 50, 100, 200,300]
}
# Create a based model
rf = RandomForestClassifier()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf,param_grid = param,scoring= 'roc_auc',
                          cv = 4, n_jobs = -1,verbose = 1)

# Fit the grid search to the data
grid_search.fit(df_train_pca,y_train)

#### printing the optimal accuracy score and hyperparameters

In [ ]:
print('We can get best score of',grid_search.best_score_,'using',grid_search.best_params_)

In [ ]:
# model with the best hyperparameters
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(bootstrap=True,
                             max_depth=20,
                             min_samples_leaf=5,
                             min_samples_split=200,
                             max_features=4,
                             n_estimators=300)

In [ ]:
# fit
rfc.fit(df_train_pca,y_train)
# predict
predictions = rfc.predict(df_test_pca)

In [ ]:
print(classification_report(y_test,predictions))

In [ ]:
# metrics
print(metrics.confusion_matrix(y_test, predictions), "\n")
print("accuracy", metrics.accuracy_score(y_test, predictions))
print("precision", metrics.precision_score(y_test, predictions))
print("sensitivity/recall", metrics.recall_score(y_test, predictions))
print("roc_auc_score", metrics.roc_auc_score(y_test, predictions))

## The classification regression model is the most effective, with a 77% recall and a ROC value of.86

## Model Building for identifying important predictor attributes which help the business understand indicators of churn

In [ ]:
#y = data_hvc.churn
#X = data_hvc.drop('churn', axis=1)
# scaling the features

#scaler = StandardScaler()
#X_scaled=scaler.fit_transform(X)
#X_scaled_df=pd.DataFrame(X_scaled,columns=X.columns)
# setting up testing and training sets
#X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, train_size=0.7, test_size=0.3, random_state=100)

In [ ]:
X_train.head()

In [ ]:
y_train.head()

In [ ]:
print(X_train.shape,y_train.shape)

#### will use RFE for feature reduction

In [ ]:
logreg = LogisticRegression()

In [ ]:
rfe = RFE(logreg,n_features_to_select=30)

In [ ]:
rfe = rfe.fit(X_train, y_train)

In [ ]:
rfe.support_

In [ ]:
list(zip(X_train.columns, rfe.support_, rfe.ranking_))

In [ ]:
col = X_train.columns[rfe.support_]

In [ ]:
X_train.columns[~rfe.support_]

##  Assessment with RandomForest Model

In [ ]:
# Running the random forest with default parameters.
rfc = RandomForestClassifier()
rfc.fit(X_train[col],y_train)

In [ ]:
# Making predictions
predictions = rfc.predict(X_test[col])

In [ ]:
# Let's check the report of our default model
print(classification_report(y_test,predictions))

### Hyperparameter Tuning

In [ ]:
# Create the parameter grid based on the results of random search
param_grid = {
    'max_depth': [8,12,16],
    'min_samples_leaf': range(100, 800, 200),
    'min_samples_split': range(200, 1000, 200),
    'n_estimators': [100,200, 300],
    'max_features': [6,9,12]
}
# Create a based model
rf = RandomForestClassifier()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid,scoring= 'roc_auc',
                          cv = 3, n_jobs = -1,verbose = 1)

# Fit the grid search to the data
grid_search.fit(X_train[col],y_train)

In [ ]:
# printing the optimal accuracy score and hyperparameters
print('We can get best score of',grid_search.best_score_,'using',grid_search.best_params_)

In [ ]:

# model with the best hyperparameters
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(bootstrap=True,
                             max_depth=16,
                             min_samples_leaf=100,
                             min_samples_split=200,
                             max_features=9,
                             n_estimators=200)

In [ ]:
# fit
rfc.fit(X_train[col],y_train)
# predict
predictions = rfc.predict(X_test[col])

In [ ]:
print(classification_report(y_test,predictions))

In [ ]:
# metrics
print(metrics.confusion_matrix(y_test, predictions), "\n")
print("accuracy", metrics.accuracy_score(y_test, predictions))
print("precision", metrics.precision_score(y_test, predictions))
print("sensitivity/recall", metrics.recall_score(y_test, predictions))
print("roc_auc_score", metrics.roc_auc_score(y_test, predictions))

#### We are getting sensitivity/Recall 61% which is very low. So we will try a different model

## Logistical Regression Model

In [ ]:
y_train.values.reshape(-1,1)

In [ ]:
X_train.reset_index(drop=True,inplace=True)
X_train.head()

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()

In [ ]:
from sklearn.feature_selection import RFE
rfe = RFE(logreg,n_features_to_select=20)
rfe = rfe.fit(X_train, y_train)

In [ ]:
rfe.support_

In [ ]:
list(zip(X_train.columns, rfe.support_, rfe.ranking_))

In [ ]:
col = X_train.columns[rfe.support_]

In [ ]:
col

In [ ]:
X_train.columns[~rfe.support_]

#### Assessing the model with StatsModels

In [ ]:
X_train.shape

In [ ]:
X_train.head()

In [ ]:
y_train.shape

In [ ]:
y_train.reset_index(drop=True,inplace=True)
y_train.head()

In [ ]:
X_train_sm = sm.add_constant(X_train[col])
logm2 = sm.GLM(y_train,X_train_sm, family = sm.families.Binomial())
res = logm2.fit()
res.summary()

In [ ]:
# Getting the predicted values on the train set
y_train_pred = res.predict(X_train_sm)
y_train_pred.shape

In [ ]:
y_train_pred = np.array(y_train_pred).reshape(-1)
y_train_pred[:10]

In [ ]:
y_train_pred_final = pd.DataFrame({'Churn':y_train, 'Churn_Prob':y_train_pred})
y_train_pred_final['CustID'] = y_train.index
y_train_pred_final.head()

In [ ]:
y_train_pred_final['predicted'] = y_train_pred_final.Churn_Prob.map(lambda x:1 if x > 0.5 else 0)

# Let's see the head
y_train_pred_final.head()

In [ ]:
# Confusion matrix
confusion = metrics.confusion_matrix(y_train_pred_final.Churn, y_train_pred_final.predicted )
print(confusion)

In [ ]:
# Let's check the overall accuracy.
print(metrics.accuracy_score(y_train_pred_final.Churn, y_train_pred_final.predicted))

####  Check for the VIF values of the feature variables.


In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
# Create a dataframe that will contain the names of all the feature variables and their respective VIFs
vif = pd.DataFrame()
vif['Features'] = X_train[col].columns
vif['VIF'] = [variance_inflation_factor(X_train[col].values, i) for i in range(X_train[col].shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

#### drop count_rech_3g_good_phase having High VIF value and recheck

In [ ]:
col = col.drop('count_rech_3g_good_phase')
col

In [ ]:
# Let's re-run the model using the selected variables
X_train_sm = sm.add_constant(X_train[col])
logm3 = sm.GLM(y_train,X_train_sm, family = sm.families.Binomial())
res = logm3.fit()
res.summary()

In [ ]:
y_train_pred = res.predict(X_train_sm).values.reshape(-1)

In [ ]:
y_train_pred_final['Churn_Prob'] = y_train_pred

In [ ]:
# Creating new column 'predicted' with 1 if Churn_Prob > 0.5 else 0
y_train_pred_final['predicted'] = y_train_pred_final.Churn_Prob.map(lambda x: 1 if x > 0.5 else 0)
y_train_pred_final.head()

In [ ]:
# Let's check the overall accuracy.
print(metrics.accuracy_score(y_train_pred_final.Churn, y_train_pred_final.predicted))


#### Let's check VIF again

In [ ]:
vif = pd.DataFrame()
vif['Features'] = X_train[col].columns
vif['VIF'] = [variance_inflation_factor(X_train[col].values, i) for i in range(X_train[col].shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

#### P value high for "monthly_3g_action_phase"  . Will drop this and remodel

In [ ]:
col = col.drop('monthly_3g_action_phase')
col

In [ ]:
# Let's re-run the model using the selected variables
X_train_sm = sm.add_constant(X_train[col])
logm4 = sm.GLM(y_train,X_train_sm, family = sm.families.Binomial())
res = logm4.fit()
res.summary()

#### Let's check VIF again

In [ ]:
vif = pd.DataFrame()
vif['Features'] = X_train[col].columns
vif['VIF'] = [variance_inflation_factor(X_train[col].values, i) for i in range(X_train[col].shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

#### sachet_3g_action_phase is having high P value, we will drop this and remodel

In [ ]:
col = col.drop('sachet_3g_action_phase')
col

In [ ]:
# Let's re-run the model using the selected variables
X_train_sm = sm.add_constant(X_train[col])
logm5 = sm.GLM(y_train,X_train_sm, family = sm.families.Binomial())
res = logm5.fit()
res.summary()

#### Now all P values are less than 0.05. Let's check VIF again

In [ ]:
vif = pd.DataFrame()
vif['Features'] = X_train[col].columns
vif['VIF'] = [variance_inflation_factor(X_train[col].values, i) for i in range(X_train[col].shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

#### count_rech_2g_good_phase is having highest VIF value. We will drop this and remodel

In [ ]:
col = col.drop('count_rech_2g_good_phase')
col

In [ ]:
# Let's re-run the model using the selected variables
X_train_sm = sm.add_constant(X_train[col])
logm6 = sm.GLM(y_train,X_train_sm, family = sm.families.Binomial())
res = logm6.fit()
res.summary()

#### Checking VIF value

In [ ]:
vif = pd.DataFrame()
vif['Features'] = X_train[col].columns
vif['VIF'] = [variance_inflation_factor(X_train[col].values, i) for i in range(X_train[col].shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

#### std_og_mou_good_phase is having high VIF, will drop it

In [ ]:
col = col.drop('std_og_mou_good_phase')
col

In [ ]:
# Let's re-run the model using the selected variables
X_train_sm = sm.add_constant(X_train[col])
logm7 = sm.GLM(y_train,X_train_sm, family = sm.families.Binomial())
res = logm7.fit()
res.summary()

In [ ]:
vif = pd.DataFrame()
vif['Features'] = X_train[col].columns
vif['VIF'] = [variance_inflation_factor(X_train[col].values, i) for i in range(X_train[col].shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

#### P value of onnet_mou_good_phase is higher than 0.05, will drop it.

In [ ]:
col = col.drop('onnet_mou_good_phase')
col

In [ ]:
# Let's re-run the model using the selected variables
X_train_sm = sm.add_constant(X_train[col])
logm8 = sm.GLM(y_train,X_train_sm, family = sm.families.Binomial())
res = logm8.fit()
res.summary()

In [ ]:
vif = pd.DataFrame()
vif['Features'] = X_train[col].columns
vif['VIF'] = [variance_inflation_factor(X_train[col].values, i) for i in range(X_train[col].shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
coef_dic = {"fb_user_8":-1.5011,"sep_vbc_3g":-2.0045,"duration_last_rech_data_8":0.7331,"offnet_mou_good_phase":0.0662,"loc_og_mou_good_phase":-1.0688,
           "loc_og_mou_action_phase":-0.7962,"spl_ic_mou_good_phase":-0.7394,"spl_ic_mou_action_phase":-0.8440,"total_ic_mou_good_phase":-1.3653,"total_ic_mou_action_phase":-1.3176,
           "count_rech_3g_action_phase":-0.1422,"sachet_2g_good_phase":0.1007,"monthly_3g_good_phase":0.1252,"sachet_3g_good_phase":0.0947}

#### Now P value of all variables are below 0.05 and VIF also less than 5
#### We will use this as final model and predict values

In [ ]:
y_train_pred = res.predict(X_train_sm).values.reshape(-1)

In [ ]:
y_train_pred_final['Churn_Prob'] = y_train_pred

In [ ]:
# Creating new column 'predicted' with 1 if Churn_Prob > 0.5 else 0
y_train_pred_final['predicted'] = y_train_pred_final.Churn_Prob.map(lambda x: 1 if x > 0.5 else 0)
y_train_pred_final.head()

In [ ]:
# Let's check the overall accuracy.
print(metrics.accuracy_score(y_train_pred_final.Churn, y_train_pred_final.predicted))

In [ ]:
# Let's take a look at the confusion matrix again
confusion = metrics.confusion_matrix(y_train_pred_final.Churn, y_train_pred_final.predicted )
confusion

In [ ]:
# Let's check the overall accuracy.
metrics.accuracy_score(y_train_pred_final.Churn, y_train_pred_final.predicted)

In [ ]:
TP = confusion[1,1] # true positive
TN = confusion[0,0] # true negatives
FP = confusion[0,1] # false positives
FN = confusion[1,0] # false negatives

In [ ]:
# Let's see the sensitivity of our logistic regression model
TP / float(TP+FN)

In [ ]:
# Let us calculate specificity
TN / float(TN+FP)

In [ ]:
# Calculate false postive rate - predicting churn when customer does not have churned
print(FP/ float(TN+FP))

In [ ]:
# positive predictive value
print (TP / float(TP+FP))

In [ ]:
# Negative predictive value
print (TN / float(TN+ FN))

#### Plot ROC curve

In [ ]:
def draw_roc( actual, probs ):
    fpr, tpr, thresholds = metrics.roc_curve( actual, probs,
                                              drop_intermediate = False )
    auc_score = metrics.roc_auc_score( actual, probs )
    plt.figure(figsize=(5, 5))
    plt.plot( fpr, tpr, label='ROC curve (area = %0.2f)' % auc_score )
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate or [1 - True Negative Rate]')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()

    return None

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve( y_train_pred_final.Churn, y_train_pred_final.Churn_Prob, drop_intermediate = False )

In [ ]:
draw_roc(y_train_pred_final.Churn, y_train_pred_final.Churn_Prob)

####  Finding Optimal Cutoff Point

In [ ]:
# Let's create columns with different probability cutoffs
numbers = [float(x)/10 for x in range(10)]
for i in numbers:
    y_train_pred_final[i]= y_train_pred_final.Churn_Prob.map(lambda x: 1 if x > i else 0)
y_train_pred_final.head()

In [ ]:
# Now let's calculate accuracy sensitivity and specificity for various probability cutoffs.
cutoff_df = pd.DataFrame( columns = ['prob','accuracy','sensi','speci'])
from sklearn.metrics import confusion_matrix

# TP = confusion[1,1] # true positive
# TN = confusion[0,0] # true negatives
# FP = confusion[0,1] # false positives
# FN = confusion[1,0] # false negatives

num = [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
for i in num:
    cm1 = metrics.confusion_matrix(y_train_pred_final.Churn, y_train_pred_final[i] )
    total1=sum(sum(cm1))
    accuracy = (cm1[0,0]+cm1[1,1])/total1

    speci = cm1[0,0]/(cm1[0,0]+cm1[0,1])
    sensi = cm1[1,1]/(cm1[1,0]+cm1[1,1])
    cutoff_df.loc[i] =[ i ,accuracy,sensi,speci]
print(cutoff_df)

In [ ]:
# Let's plot accuracy sensitivity and specificity for various probabilities.
cutoff_df.plot.line(x='prob', y=['accuracy','sensi','speci'])
plt.show()

#### select 0.1 as cutoff

In [ ]:
y_train_pred_final['final_predicted'] = y_train_pred_final.Churn_Prob.map( lambda x: 1 if x > 0.60 else 0)

y_train_pred_final.head()


In [ ]:
# Let's check the overall accuracy.
metrics.accuracy_score(y_train_pred_final.Churn, y_train_pred_final.final_predicted)

In [ ]:
confusion2 = metrics.confusion_matrix(y_train_pred_final.Churn, y_train_pred_final.final_predicted )
confusion2

In [ ]:
TP = confusion2[1,1] # true positive
TN = confusion2[0,0] # true negatives
FP = confusion2[0,1] # false positives
FN = confusion2[1,0] # false negatives

In [ ]:
# Let's see the sensitivity of our logistic regression model
TP / float(TP+FN)

In [ ]:
# Let us calculate specificity
TN / float(TN+FP)

In [ ]:
# Calculate false postive rate - predicting churn when customer does not have churned
print(FP/ float(TN+FP))

In [ ]:
# Positive predictive value
print (TP / float(TP+FP))

In [ ]:
# Negative predictive value
print (TN / float(TN+ FN))

#### Logical Regression appears to be a superior option due to its high sensitivity (82%), positive predictive value (0.82%), and negative predictive value (0.83%)

##  Now we will use this model to test on Test set

In [ ]:
X_test.head()

In [ ]:
X_test_sm = sm.add_constant(X_test[col])

#### Making predictions on Test set

In [ ]:
y_test_pred = res.predict(X_test_sm)

In [ ]:
# Converting y_pred to a dataframe which is an array
y_pred_1 = pd.DataFrame(y_test_pred)

In [ ]:
# Let's see the head
y_pred_1.head()

In [ ]:
# Converting y_test to dataframe
y_test_df = pd.DataFrame(y_test)

In [ ]:
# Putting CustID to index
y_test_df['CustID'] = y_test_df.index

In [ ]:
# Removing index for both dataframes to append them side by side
y_pred_1.reset_index(drop=True, inplace=True)
y_test_df.reset_index(drop=True, inplace=True)

In [ ]:
# Appending y_test_df and y_pred_1
y_pred_final = pd.concat([y_test_df, y_pred_1],axis=1)

In [ ]:
y_pred_final.head()

In [ ]:
# Renaming the column
y_pred_final= y_pred_final.rename(columns={ 0 : 'Churn_Prob'})

In [ ]:
y_pred_final.head()

In [ ]:
y_pred_final['final_predicted'] = y_pred_final.Churn_Prob.map(lambda x: 1 if x > 0.59 else 0)

In [ ]:
y_pred_final.head()

In [ ]:
# Let's check the overall accuracy.
metrics.accuracy_score(y_pred_final.churn, y_pred_final.final_predicted)

In [ ]:
confusion2 = metrics.confusion_matrix(y_pred_final.churn, y_pred_final.final_predicted )
confusion2

In [ ]:
TP = confusion2[1,1] # true positive
TN = confusion2[0,0] # true negatives
FP = confusion2[0,1] # false positives
FN = confusion2[1,0] # false negatives

In [ ]:
# Let's see the sensitivity of our logistic regression model
TP / float(TP+FN)

In [ ]:
# Let us calculate specificity
TN / float(TN+FP)

### Feature Importance

In [ ]:
out = dict(sorted(coef_dic.items(), key=lambda t: abs(t[1]),reverse=True))
out

In [ ]:
out_df = pd.DataFrame(out.items(), columns=['Feature', 'Coef'])

In [ ]:
out_df

#### We will plot the feature and Coef to understand the influence of each features on churn of the customers

In [ ]:
plt.figure(figsize=(15,9))
sns.barplot(x=out_df.Feature,y=out_df.Coef)
plt.xticks(rotation=45)
plt.show()

#### We will plot Coefficiants with its absolute values for finding the top 5 features

In [ ]:
plt.figure(figsize=(15,9))
sns.barplot(x=out_df.Feature,y=abs(out_df.Coef))
plt.xticks(rotation= 75)
plt.show()

### The top5 Features which can influence the customer Churn are :
### 1. sep_vbc_3g
- Volume based cost - when no specific scheme is not purchased and paid as per usage for 3g in the month of September
- This has negative coefficant meaning inversly correlated with churn.
- Means the more the value, lesser the chance of churn
### 2. fb_user_8
- Service scheme to avail services of Facebook and similar social networking sites on 8th month
- This has negative coefficant meaning inversly correlated with churn.
- Means the more the value, lesser the chance of churn
### 3. total_ic_mou_good_phase
- Total Incoming Calls Minutes of usage - voice calls during the Good phase (6&7 months)
- This has negative coefficant meaning inversly correlated with churn.
- Means the more the value, lesser the chance of churn
### 4. total_ic_mou_action_phase
- Total Incoming Calls Minutes of usage - voice calls during the Action phase (8th month)
- This has negative coefficant meaning inversly correlated with churn.
- Means the more the value, lesser the chance of churn
### 5. loc_og_mou_good_phase
- Local calls - within same telecom circle out going calls Minutes of usage - voice calls during the Good phase (6&7) months
- This has negative coefficant meaning inversly correlated with churn.
- Means the more the value, lesser the chance of churn

# Recomendations

### 1.  Offering specific packages or themes for users to access popular services like Facebook, Instagram, and WhatsApp can help decrease the churn rate.

### 2.  The introduction of competitive rates for 3g data can potentially decrease customer churn.

### 3. Introducing additional call package categories can increase customer acquisition and decrease churn.